In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

In [3]:
        df = sns.load_dataset('exercise')
        df.head()

,Unnamed: 0,id,diet,pulse,time,kind
0,0,1,low fat,85,1 min,rest
1,1,1,low fat,85,15 min,rest
2,2,1,low fat,88,30 min,rest
3,3,2,low fat,90,1 min,rest
4,4,2,low fat,92,15 min,rest


In [4]:
df.shape

(90, 6)

## Dicionário ##

In [5]:
dicionario = pd.DataFrame([
    {

        'variável': 'Unnamed: 0',
        'descricao': 'contagem',
        'tipo': 'quantitativa',
        'subtipo': 'continua'
    },{
        'variável': 'id',
        'descricao': 'identificação',
        'tipo': 'quantitativa',
        'subtipo': 'discreta'
    },{
        'variável': 'diet',
        'descricao': 'dieta dos animais',
        'tipo': 'qualitativa',
        'subtipo': 'nominal'
    },{
        'variável': 'pulse',
        'descricao': 'batimentos cardíacos',
        'tipo': 'quantitativa',
        'subtipo': 'discreta'
    },{
        'variável': 'time',
        'descricao': 'tempo de medição',
        'tipo': 'quantitativa',
        'subtipo': 'discreta'
    },{
        'variável': 'kind',
        'descricao': 'estado do animal na medição',
        'tipo': 'qualitativa',
        'subtipo': 'nominal'
    }
])

dicionario

,variável,descricao,tipo,subtipo
0,Unnamed: 0,contagem,quantitativa,continua
1,id,identificação,quantitativa,discreta
2,diet,dieta dos animais,qualitativa,nominal
3,pulse,batimentos cardíacos,quantitativa,discreta
4,time,tempo de medição,quantitativa,discreta
5,kind,estado do animal na medição,qualitativa,nominal


## Limpeza de dados ##


In [6]:
df.reset_index(inplace = True, drop = True)
df

,Unnamed: 0,id,diet,pulse,time,kind
0,0,1,low fat,85,1 min,rest
1,1,1,low fat,85,15 min,rest
2,2,1,low fat,88,30 min,rest
3,3,2,low fat,90,1 min,rest
4,4,2,low fat,92,15 min,rest
...,...,...,...,...,...,...
85,85,29,no fat,135,15 min,running
86,86,29,no fat,130,30 min,running
87,87,30,no fat,99,1 min,running
88,88,30,no fat,111,15 min,running


A coluna "Unnamed" estava servindo apenas como contador, assim como a coluna "id", por isso foi retirada.

In [7]:
df.drop('Unnamed: 0', axis=1, inplace = True)

df

,id,diet,pulse,time,kind
0,1,low fat,85,1 min,rest
1,1,low fat,85,15 min,rest
2,1,low fat,88,30 min,rest
3,2,low fat,90,1 min,rest
4,2,low fat,92,15 min,rest
...,...,...,...,...,...
85,29,no fat,135,15 min,running
86,29,no fat,130,30 min,running
87,30,no fat,99,1 min,running
88,30,no fat,111,15 min,running


## Tratamento de dados faltantes. ##

In [8]:
ausencia_values = df.isnull().sum()
print(ausencia_values)

id       0
diet     0
pulse    0
time     0
kind     0
dtype: int64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id      90 non-null     int64   
 1   diet    90 non-null     category
 2   pulse   90 non-null     int64   
 3   time    90 non-null     category
 4   kind    90 non-null     category
dtypes: category(3), int64(2)
memory usage: 2.2 KB


Não há dados faltante nesse dataset

## Tratamento de dados discrepantes ##

In [10]:
numeric_data = df.select_dtypes(include=['int64', 'float64'])

Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1
outliers = numeric_data[(numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))]

print(outliers)

    id  pulse
0  NaN    NaN
1  NaN    NaN
2  NaN    NaN
3  NaN    NaN
4  NaN    NaN
..  ..    ...
85 NaN  135.0
86 NaN  130.0
87 NaN    NaN
88 NaN    NaN
89 NaN  150.0

[90 rows x 2 columns]


In [11]:
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
outliers = numeric_data[(numeric_data < limite_inferior)| (numeric_data > limite_superior)]

print(outliers)

    id  pulse
0  NaN    NaN
1  NaN    NaN
2  NaN    NaN
3  NaN    NaN
4  NaN    NaN
..  ..    ...
85 NaN  135.0
86 NaN  130.0
87 NaN    NaN
88 NaN    NaN
89 NaN  150.0

[90 rows x 2 columns]


In [12]:
dados_limpos = numeric_data[(numeric_data >= limite_inferior) & (numeric_data <= limite_superior)]

print(dados_limpos)

    id  pulse
0    1   85.0
1    1   85.0
2    1   88.0
3    2   90.0
4    2   92.0
..  ..    ...
85  29    NaN
86  29    NaN
87  30   99.0
88  30  111.0
89  30    NaN

[90 rows x 2 columns]


In [13]:
#dados_limpos['pulse'].fillna(dados_limpos['pulse'].median(), inplace=True)
#print(dados_limpos)

## Transformação de dados ##

## Variáveis Nominais ##

One-Hot Enconding: Transforma categorias em colunas binárias. Útil quando não há ordem entre as categorias.

In [14]:
dicionario

,variável,descricao,tipo,subtipo
0,Unnamed: 0,contagem,quantitativa,continua
1,id,identificação,quantitativa,discreta
2,diet,dieta dos animais,qualitativa,nominal
3,pulse,batimentos cardíacos,quantitativa,discreta
4,time,tempo de medição,quantitativa,discreta
5,kind,estado do animal na medição,qualitativa,nominal


In [14]:
dicionario = pd.get_dummies(dicionario, columns = ['tipo'])
dicionario

,variável,descricao,subtipo,tipo_qualitativa,tipo_quantitativa
0,Unnamed: 0,contagem,continua,False,True
1,id,identificação,discreta,False,True
2,diet,dieta dos animais,nominal,True,False
3,pulse,batimentos cardíacos,discreta,False,True
4,time,tempo de medição,discreta,False,True
5,kind,estado do animal na medição,nominal,True,False


In [15]:
dicionario.describe()

,variável,descricao,tipo,subtipo
count,6,6,6,6
unique,6,6,2,3
top,Unnamed: 0,contagem,quantitativa,discreta
freq,1,1,4,3


## Variável Ordinal: ##

Ordinal Encoder é usado quando existe uma ordem natural entre as categorias. Ele atribui valores inteiros sequenciais às categorias, preservando a relação de ordem.

In [16]:
ordem_subtipo = ['continua', 'discreta', 'nominal']

oe = OrdinalEncoder(categories=[ordem_subtipo])
dicionario['subtipo'] = oe.fit_transform(dicionario[['subtipo']])

dicionario

,variável,descricao,tipo,subtipo
0,Unnamed: 0,contagem,quantitativa,0.0
1,id,identificação,quantitativa,1.0
2,diet,dieta dos animais,qualitativa,2.0
3,pulse,batimentos cardíacos,quantitativa,1.0
4,time,tempo de medição,quantitativa,1.0
5,kind,estado do animal na medição,qualitativa,2.0


## Variável Temporal ##

Extrair componentes e adicionar colunas cíclicas.

In [17]:
df


,id,diet,pulse,time,kind
0,1,low fat,85,1 min,rest
1,1,low fat,85,15 min,rest
2,1,low fat,88,30 min,rest
3,2,low fat,90,1 min,rest
4,2,low fat,92,15 min,rest
...,...,...,...,...,...
85,29,no fat,135,15 min,running
86,29,no fat,130,30 min,running
87,30,no fat,99,1 min,running
88,30,no fat,111,15 min,running


In [18]:
df['time'] = pd.to_datetime(df['time'], format="%M min")
df['Minuto'] = df['time'].dt.minute
df = df.drop(columns=['time'])
df

,id,diet,pulse,kind,Minuto
0,1,low fat,85,rest,1
1,1,low fat,85,rest,15
2,1,low fat,88,rest,30
3,2,low fat,90,rest,1
4,2,low fat,92,rest,15
...,...,...,...,...,...
85,29,no fat,135,running,15
86,29,no fat,130,running,30
87,30,no fat,99,running,1
88,30,no fat,111,running,15
